In [ ]:
!pip install git+https://github.com/jxx123/simglucose.git
!pip install d3rlpy


In [ ]:
import numpy as np
from datetime import datetime

from simglucose.patient.t1dpatient import T1DPatient
from simglucose.sensor.cgm import CGMSensor
from simglucose.actuator.pump import InsulinPump
from simglucose.simulation.env import T1DSimEnv
from simglucose.simulation.scenario_gen import RandomScenario
from simglucose.analysis.risk import risk_index



REWARD_HORIZON = 24   # last 2 hours (24 * 5min = 120min)
TMAX_STEPS     = 2500
EARLY_PENALTY_ALPHA = 1.2
EARLY_PENALTY_WORST_STEP = 100.0

def full_risk_index_reward(bg_history, horizon=REWARD_HORIZON):
    
    if len(bg_history) < horizon:
        return 0.0
    _, _, RI = risk_index(bg_history[-horizon:], horizon)
    return -float(RI)



def make_env(patient_id="adult#001", seed=42):
    patient = T1DPatient.withName(patient_id)
    sensor  = CGMSensor.withName("Dexcom")
    pump    = InsulinPump.withName("Insulet")

    scen = RandomScenario(start_time=datetime.now(), seed=seed)
    env  = T1DSimEnv(patient=patient, sensor=sensor, pump=pump, scenario=scen)
    env.sample_time = 5  # minutes
    return env



def obs_to_state(obs, info, t_minute):
    
    bg   = float(info["bg"])
    meal = float(info["meal"])
    iob  = float(info["patient_state"][5])  
    time_of_day = float(t_minute % (24*60)) / (24*60.0)   

    return np.array([bg, meal, iob, time_of_day], dtype=np.float32)

def compute_tir_tbr_cv(bg_series, low=70.0, high=180.0):
    bg = np.asarray(bg_series, dtype=float)
    valid = ~np.isnan(bg)
    bg = bg[valid]
    if len(bg) == 0:
        return 0.0, 0.0, 0.0

    tir = np.mean((bg >= low) & (bg <= high)) * 100.0
    tbr = np.mean(bg < low) * 100.0
    cv  = np.std(bg) / max(np.mean(bg), 1e-6) * 100.0
    return float(tir), float(tbr), float(cv)


In [ ]:
from simglucose.controller.base import Action
from simglucose.patient.t1dpatient import T1DPatient
from simglucose.sensor.cgm import CGMSensor
from simglucose.actuator.pump import InsulinPump
from simglucose.simulation.env import T1DSimEnv
from simglucose.simulation.scenario_gen import RandomScenario
from simglucose.analysis.risk import risk_index

from d3rlpy.dataset import MDPDataset
import numpy as np
from datetime import datetime

REWARD_HORIZON = 24   
TMAX_STEPS     = 2500

def full_risk_index_reward(bg_history, horizon=REWARD_HORIZON):
    if len(bg_history) < horizon:
        return 0.0
    _, _, RI = risk_index(bg_history[-horizon:], horizon)
    return -float(RI)

def make_env(patient_id="adult#001", seed=42):
    patient = T1DPatient.withName(patient_id)
    sensor  = CGMSensor.withName("Dexcom")
    pump    = InsulinPump.withName("Insulet")
    scen    = RandomScenario(start_time=datetime.now(), seed=seed)
    env     = T1DSimEnv(patient=patient, sensor=sensor, pump=pump, scenario=scen)
    env.sample_time = 5
    return env

def obs_to_state(obs, info, t_minute):
    bg   = float(info["bg"])
    meal = float(info["meal"])
    iob  = float(info["patient_state"][5])
    time_of_day = float(t_minute % (24 * 60)) / (24 * 60.0)
    return np.array([bg, meal, iob, time_of_day], dtype=np.float32)

def heuristic_policy(bg, meal, iob):
    base = 0.05
    meal_bonus = 0.01 * (meal > 0)
    high_bg_bonus = 0.0005 * max(bg - 150, 0)
    return float(base + meal_bonus + high_bg_bonus)

def collect_offline_dataset(
    n_episodes=80,
    patient_id="adult#001",
    seed=42,
    max_steps=TMAX_STEPS,
    gamma=0.95
):
    rng = np.random.RandomState(seed)

    observations      = []
    actions           = []
    rewards           = []
    next_observations = []
    terminals         = []

    for ep in range(n_episodes):
        env = make_env(patient_id=patient_id, seed=seed + ep)
        step_result = env.reset()
        obs = step_result.observation
        info = step_result.info

        bg_history = [info["bg"]]
        t_minute = 0
        done = False
        step = 0

        print(f"[Dataset] Episode {ep+1}/{n_episodes}")

        while not done and step < max_steps:
            step += 1
            t_minute += 5

            bg   = float(info["bg"])
            meal = float(info["meal"])
            iob  = float(info["patient_state"][5])

            state = obs_to_state(obs, info, t_minute)

       
            act_value = heuristic_policy(bg, meal, iob)
            act_value += rng.normal(0.0, 0.01)
            act_value = max(0.0, act_value)

            
            sim_action = Action(basal=0.0, bolus=act_value)

            step_result = env.step(sim_action)
            next_obs  = step_result.observation
            done_env  = step_result.done
            info      = step_result.info

            new_bg = float(info["bg"])
            bg_history.append(new_bg)

            # reward
            r = full_risk_index_reward(bg_history, horizon=REWARD_HORIZON)

            time_limit = (step >= max_steps)
            terminal = done_env or time_limit

            if done_env and not time_limit:
                worst_step = 100.0
                alpha      = 1.2
                remaining  = max_steps - step
                if remaining > 0:
                    r += -alpha * worst_step * remaining

            next_state = obs_to_state(next_obs, info, t_minute)

            observations.append(state)
            actions.append([act_value])
            rewards.append(r)
            next_observations.append(next_state)
            terminals.append(terminal)

            obs = next_obs
            done = terminal

    observations      = np.asarray(observations, dtype=np.float32)
    actions           = np.asarray(actions, dtype=np.float32)
    rewards           = np.asarray(rewards, dtype=np.float32)
    next_observations = np.asarray(next_observations, dtype=np.float32)
    terminals         = np.asarray(terminals, dtype=bool)

    dataset = MDPDataset(
        observations=observations,
        actions=actions,
        rewards=rewards,
        terminals=terminals
    )
    return dataset


dataset = collect_offline_dataset(
    n_episodes= 5, #80,
    patient_id="adult#001",
    seed=123
)
print("Dataset size:", dataset.observations.shape)


1. BCQ


In [ ]:
from d3rlpy.algos import BCQ, BCQConfig

bcq_config = BCQConfig(
    gamma=0.95,
    batch_size=256,
)

bcq = BCQ(bcq_config, device="cpu", enable_ddp=False)



bcq.fit(
    dataset,
    n_steps=2500,
    n_steps_per_epoch=10_000,
    save_interval=0,
)



print("BCQ training done.")


2. TD3


In [ ]:
from d3rlpy.algos import TD3PlusBC, TD3PlusBCConfig

td3_config = TD3PlusBCConfig(
    gamma=0.95,
    alpha=2.5,         
    batch_size=256,
)

td3_bc = td3_config.create(
    device="cpu",
    enable_ddp=False,
)

td3_bc.fit(
    dataset,
    n_steps=2500,
    n_steps_per_epoch=10_000,
    save_interval=0,
)

print("TD3-BC training done.")


In [ ]:
from simglucose.controller.base import Action
import numpy as np

def compute_tir_tbr_cv(bg_series, low=70.0, high=180.0):
    bg = np.asarray(bg_series, dtype=float)
    if len(bg) == 0:
        return 0.0, 0.0, 0.0
    tir = np.mean((bg >= low) & (bg <= high)) * 100.0
    tbr = np.mean(bg < low) * 100.0
    cv  = np.std(bg) / max(np.mean(bg), 1e-6) * 100.0
    return float(tir), float(tbr), float(cv)


def evaluate_policy(algo, n_episodes=10, patient_id="adult#001", seed=999):
    all_tir, all_tbr, all_cv = [], [], []

    for ep in range(n_episodes):
        env = make_env(patient_id=patient_id, seed=seed + ep)
        step_result = env.reset()
        obs  = step_result.observation
        info = step_result.info

        t_minute = 0
        done = False
        step = 0

        bg_traj = []
        bg_traj.append(float(info["bg"]))

        while not done and step < TMAX_STEPS:
            step += 1
            t_minute += 5

            
            state = obs_to_state(obs, info, t_minute)          
            state_vec = state.reshape(1, -1)                   

            
            action_arr = algo.predict(state_vec)               
            act_value = float(action_arr[0].item())            

            
            sim_action = Action(basal=0.0, bolus=max(0.0, act_value))

            step_result = env.step(sim_action)
            obs  = step_result.observation
            info = step_result.info

            bg = float(info["bg"])
            bg_traj.append(bg)

            done = step_result.done or (step >= TMAX_STEPS)

        tir, tbr, cv = compute_tir_tbr_cv(bg_traj)
        print(f"[Eval] Episode {ep+1}/{n_episodes}  TIR={tir:.1f}  TBR={tbr:.1f}  CV={cv:.1f}")
        all_tir.append(tir)
        all_tbr.append(tbr)
        all_cv.append(cv)

    return float(np.mean(all_tir)), float(np.mean(all_tbr)), float(np.mean(all_cv))


bcq_tir, bcq_tbr, bcq_cv = evaluate_policy(bcq, n_episodes=10)
td3_tir, td3_tbr, td3_cv = evaluate_policy(td3_bc, n_episodes=10)

print("\nBCQ    -> TIR={:.1f}  TBR={:.1f}  CV={:.1f}".format(bcq_tir, bcq_tbr, bcq_cv))
print("TD3-BC -> TIR={:.1f}  TBR={:.1f}  CV={:.1f}".format(td3_tir, td3_tbr, td3_cv))
